In [ ]:
import wasmtime
import sys

# --- Engine + Store setup ---
config = wasmtime.Config()
config.debug_info = True
config.wasm_backtrace = True
engine = wasmtime.Engine(config)
store = wasmtime.Store(engine)

# --- Import functions (match your module types: type19=(i64), type20=(i64,i64)) ---
def log_utf8_caller(caller, ptr, length):
    memory = caller.get_export('memory')
    assert isinstance(memory, wasmtime.Memory)
    # memory.data_ptr(store) returns a buffer-like object; slice it
    data = memory.data_ptr(store)[ptr:ptr + length]
    print("[env::log_utf8] ->", data.decode('utf-8', errors='replace'))

def input_caller(register_id):
    # type19: (param i64) -> void
    print("[env::input] called with register_id =", register_id)
    # If your WASM expects data to be placed into its register, you'd write to memory & return a register id,
    # but you told me this import has no return value, so keep it void.
    # Do whatever host-side logic is required here.

def value_return_caller(ptr, length):
    # type20: (param i64 i64) -> void
    print("[env::value_return] called with ptr =", ptr, "length =", length)

# --- Linker + define imports ---
linker = wasmtime.Linker(engine)

linker.define(
    store=store,
    module="env",
    name="log_utf8",
    item=wasmtime.Func(
        store,
        wasmtime.FuncType([wasmtime.ValType.i64(), wasmtime.ValType.i64()], []),
        log_utf8_caller
    )
)

linker.define(
    store=store,
    module="env",
    name="input",
    item=wasmtime.Func(
        store,
        wasmtime.FuncType([wasmtime.ValType.i64()], []),
        input_caller
    )
)

linker.define(
    store=store,
    module="env",
    name="value_return",
    item=wasmtime.Func(
        store,
        wasmtime.FuncType([wasmtime.ValType.i64(), wasmtime.ValType.i64()], []),
        value_return_caller
    )
)

# --- Helper utilities ---
def describe_module_exports(module):
    print("\nModule Exports (name -> type description):")
    for exp in module.exports:
        # exp is an ExportType; exp.kind is a wasmtime.EXTERN_FUNC/.. ; exp.name is the name
        k = exp.type
        # try to print function param/result counts if it's a function
        if exp.type and hasattr(exp.type, "func_type"):
            ft = exp.type.func_type
            params = [str(p) for p in ft.params]
            results = [str(r) for r in ft.results]
            print(f" - {exp.name}: func(params={params}, results={results})")
        else:
            print(f" - {exp.name}: {exp.type}")

def pretty_print_export_types(instance):
    print("\nResolved Exports (from instantiated instance):")
    for name, export in instance.exports(store).items():
        t = type(export)
        print(f" - {name}: {t}")

def try_call(func, args):
    try:
        res = func(*args)   # wasmtime.Func is callable with just args
        return ("ok", res)
    except wasmtime.Trap as e:
        return ("trap", e)
    except TypeError as e:
        return ("typeerror", e)
    except Exception as e:
        return ("error", e)

In [ ]:
import wasmtime

engine = wasmtime.Engine()
store = wasmtime.Store(engine)
linker = wasmtime.Linker(engine)

wasm_path = "test.wasm"

# Load and describe module
try:
    with open(wasm_path, "rb") as f:
        module = wasmtime.Module(engine, f.read())

    print("✅ Module loaded successfully!\n")

    print("=== WASM imports required ===")
    for imp in module.imports:
        try:
            print(f" - {imp.module}::{imp.name}  ({imp.type})")
        except AttributeError:
            # For older wasmtime bindings
            print(f" - {imp.module_name if hasattr(imp, 'module_name') else '?'}::{imp.name}")

    print("\n=== WASM exports available ===")
    for exp in module.exports:
        print(f" - {exp.name}: {exp.type}")

except Exception as e:
    print(f"❌ Failed to load/inspect WASM: {e}")


In [17]:
from wasmtime import Engine, Store, Linker, Func, FuncType, ValType, Module

engine = Engine()
store = Store(engine)
linker = Linker(engine)

wasm_path = "test.wasm"
with open(wasm_path, "rb") as f:
    module = Module(engine, f.read())

# === Correct imports based on your WASM ===
#   (import "env" "input" (func (param i64)))
#   (import "env" "log_utf8" (func (param i64 i64)))
#   (import "env" "value_return" (func (param i64 i64)))

# 1️⃣ env::input — takes one i64 param, returns nothing
def input_stub(arg):
    print(f"[host] env::input({arg}) called")

linker.define(
    store,
    "env",
    "input",
    Func(store, FuncType([ValType.i64()], []), input_stub)
)

# 2️⃣ env::log_utf8 — takes two i64 params, no return
def log_utf8_stub(ptr, length):
    print(f"[host] env::log_utf8(ptr={ptr}, len={length})")

linker.define(
    store,
    "env",
    "log_utf8",
    Func(store, FuncType([ValType.i64(), ValType.i64()], []), log_utf8_stub)
)

# 3️⃣ env::value_return — takes two i64 params, no return
def value_return_stub(ptr, length):
    print(f"[host] env::value_return(ptr={ptr}, len={length})")

linker.define(
    store,
    "env",
    "value_return",
    Func(store, FuncType([ValType.i64(), ValType.i64()], []), value_return_stub)
)

# === Instantiate module ===
try:
    instance = linker.instantiate(store, module)
    print("✅ WASM module instantiated successfully!\n")

    exports = instance.exports(store)
    print("Exports available:")
    for name, val in exports.items():
        print(f" - {name}: {type(val)}")

except Exception as e:
    print(f"❌ Failed to instantiate module: {e}")




✅ WASM module instantiated successfully!

Exports available:
 - memory: <class 'wasmtime._memory.Memory'>
 - _initialize: <class 'wasmtime._func.Func'>
 - helloworldnative: <class 'wasmtime._func.Func'>
 - returnvaluenative: <class 'wasmtime._func.Func'>
 - helloworld: <class 'wasmtime._func.Func'>
 - returnvalue: <class 'wasmtime._func.Func'>
 - cabi_realloc: <class 'wasmtime._func.Func'>


In [19]:
# Helper to safely call a WASM function
def try_call(func, args):
    try:
        result = func(store, *args)
        return ("ok", result)
    except TypeError as te:
        return ("typeerror", te)
    except wasmtime.Trap as trap:
        return ("trap", trap.message)
    except Exception as e:
        return ("error", e)

# Gather your exports
exports = instance.exports(store)

# 1️⃣ Targeted calls for known exports
to_try = [
    ("helloworld", []),
    ("helloworldnative", []),
    ("returnvalue", []),
    ("returnvaluenative", []),
]

print("\n=== Targeted export calls ===")
for name, args in to_try:
    func = exports.get(name)
    if not func:
        print(f" - {name}: not exported")
        continue

    print(f"\nCalling `{name}` with args={args} ...")
    r = try_call(func, args)
    if r[0] == "ok":
        print(f"  ✅ Success -> return = {r[1]}")
    elif r[0] == "trap":
        print(f"  ⚠️ Trap -> {r[1]}")
    else:
        print(f"  ❌ Error -> {r[1]}")

# 2️⃣ Optional: best-effort calls for all exported functions
print("\n=== Best-effort calls for all function exports ===")
for name, item in exports.items():
    if not isinstance(item, wasmtime.Func):
        continue  # skip memory/globals/tables

    print(f"\nAttempting `{name}` ...")
    # Try no-arg call first
    r = try_call(item, [])
    if r[0] == "ok":
        print(f"  ✅ Success with no args -> {r[1]}")
        continue
    if r[0] == "trap":
        print(f"  ⚠️ Trap with no args -> {r[1]}")
        continue

    # Try with 1–4 zero args as a guess
    success = False
    for n in range(1, 5):
        guess = [0] * n
        r2 = try_call(item, guess)
        if r2[0] == "ok":
            print(f"  ✅ Success with {n} zeros -> {r2[1]}")
            success = True
            break
        if r2[0] == "trap":
            print(f"  ⚠️ Trap with {n} zeros -> {r2[1]}")
            success = True
            break
    if not success:
        print("  ❌ Could not safely call this function.")



=== Targeted export calls ===

Calling `helloworld` with args=[] ...
[host] env::input(9223372036854775805) called
[host] env::log_utf8(ptr=11, len=1215200)
  ✅ Success -> return = None

Calling `helloworldnative` with args=[] ...
[host] env::input(9223372036854775805) called
[host] env::log_utf8(ptr=11, len=2227392)
  ✅ Success -> return = None

Calling `returnvalue` with args=[] ...
[host] env::input(9223372036854775805) called
[host] env::value_return(ptr=11, len=1215200)
  ✅ Success -> return = None

Calling `returnvaluenative` with args=[] ...
[host] env::input(9223372036854775805) called
[host] env::value_return(ptr=11, len=2227392)
  ✅ Success -> return = None

=== Best-effort calls for all function exports ===

Attempting `_initialize` ...
  ⚠️ Trap with no args -> error while executing at wasm backtrace:
    0:   0x11a0 - <unknown>!<wasm function 4>

Caused by:
    wasm trap: wasm `unreachable` instruction executed

Attempting `helloworldnative` ...
[host] env::input(92233720